# Process Data

## Load Libraries

In [5]:
import sys
import pandas as pd
from sqlalchemy import create_engine

## Load Data

In [8]:
def load_data(messages_filepath, categories_filepath):
    """
    Input: 
        1. messages_filepath: path of messages datasets
        2. categories_filepath: path of categories datasets
    
    Output:
        1. df: merged dataframe, which contains data from messages, categories files
          
    Process: 
        1. Load the required datasets, messages, categories
        2. Merge the two datasets
    """
    
    # Load messages dataset
    messages = pd.read_csv('messages.csv')
    
    # Load categories dataset
    categories = pd.read_csv('categories.csv')
    
    # Merge datasets
    df = pd.merge(messages, categories, on='id', how='inner')
    
    # Return dataframe
    return df

## Clean Data

In [10]:
def clean_data(df):
    """
    Input: 
        1. df: the merged dataframed created by the load_data function
    
    Output:
        1. df: cleaned dataframed after processed through the below process
    
    Process:
        1. Create Y columns
        2. Rename Y columns
        3. Clean Y columns values
        4. Replace original categories columns with the newly generated ones
        5. Check and remove duplicated row if there is
    
    """
    # 1. Create a dataframe of the 36 individual category columns
    categories = df.categories.str.split(';', expand=True)
    
    # 2. Generate Y columns name and rename columns
    category_colnames = [i[:-2] for i in categories.iloc[0, :]]
    categories.columns = category_colnames
    
    # 3. Clean columns values
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].str[-1]

        # convert column from string to numeric
        categories[column] = pd.to_numeric(categories[column], errors='coerce')
        
    # 4. Use new categories to replace the original column from `df`
    df.drop(['categories'], axis=1, inplace=True)
    df = pd.concat([df, categories], axis=1)
    
    # 5. Check number of duplicates
    if df.duplicated().sum() != 0:
        df = df.drop_duplicates()
        
    return df

## Save Data

In [11]:
def save_data(df, database_filename):
    """
    Input:
        1. df: the cleaned dataframe, which is generated by the clean data function
        2. database_filename: name of the database file
    Output:
        Saved database file
        
    Process:
        1. Create engine for saving dataframe
        2. Use to_sql method to dump dataframe to file
    """
    
    engine = create_engine('sqlite:///{}.db'.formate(database_filename))
    df.to_sql(database_filename, engine, index=False)

## Main

In [12]:
def main():
    if len(sys.argv) == 4:

        messages_filepath, categories_filepath, database_filepath = sys.argv[1:]

        print('Loading data...\n    MESSAGES: {}\n    CATEGORIES: {}'
              .format(messages_filepath, categories_filepath))
        df = load_data(messages_filepath, categories_filepath)

        print('Cleaning data...')
        df = clean_data(df)
        
        print('Saving data...\n    DATABASE: {}'.format(database_filepath))
        save_data(df, database_filepath)
        
        print('Cleaned data saved to database!')
    
    else:
        print('Please provide the filepaths of the messages and categories '\
              'datasets as the first and second argument respectively, as '\
              'well as the filepath of the database to save the cleaned data '\
              'to as the third argument. \n\nExample: python process_data.py '\
              'disaster_messages.csv disaster_categories.csv '\
              'DisasterResponse.db')

## Call Main

In [13]:
if __name__ == '__main__':
    main()

Please provide the filepaths of the messages and categories datasets as the first and second argument respectively, as well as the filepath of the database to save the cleaned data to as the third argument. 

Example: python process_data.py disaster_messages.csv disaster_categories.csv DisasterResponse.db
